In [1]:
import requests # type: ignore
from urllib.parse import urljoin
import pandas as pd # type: ignore
import schedule # type: ignore
import time
from datetime import datetime, timedelta
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.base import MIMEBase
from email import encoders
from dotenv import load_dotenv
import re

In [2]:
load_dotenv(override=True)
api_username = os.getenv('API_USERNAME')
api_password = os.getenv('API_PASSWORD')
api_url = os.getenv("API_URL")
envio_para = os.getenv('ENVIO_PARA')
smtp_server = os.getenv("EMAIL_SERVER")
email_port = int(os.getenv("EMAIL_PORT"))
email_usuario = os.getenv("EMAIL_USERNAME").strip()
email_senha = os.getenv("EMAIL_PASSWORD").strip()
destinatario_hmcg = os.getenv('DESTINATARIO_HMCG').strip()
destinatario_rocio = os.getenv('DESTINATARIO_ROCIO').strip()
destinatario_scs = os.getenv('DESTINATARIO_SCS').strip()
destinatario_iir = os.getenv('DESTINATARIO_IIR').strip()
destinatario_vivi = os.getenv('DESTINATARIO_VIVIANE').strip()
destinatario_capibaribe= os.getenv('DESTINATARIO_CAPIBERIBE').strip()

In [3]:
css_hover = """
<style>
    table {
        border-collapse: collapse;
        width: 100%;
        font-family: Arial, sans-serif;
        font-size: 14px;
        border-radius: 8px;
        overflow: hidden; /* Para manter o arredondamento no hover */
        box-shadow: 0 2px 8px rgba(0,0,0,0.1);
    }
    th, td {
        padding: 10px 14px;
        text-align: left;
        border-bottom: 1px solid #ddd;
    }
    th {
        background-color: #007bff;
        color: white;
        font-weight: 600;
    }
    tr:nth-child(even) {
        background-color: #f9f9f9;
    }
    tr:hover {
        background-color: #eaf3ff;
        transition: background-color 0.2s ease-in-out;
    }
</style>
"""

In [5]:
auth_url = urljoin(api_url, "/sessions")
body = {
    "nome": api_username,
    "password":api_password
}

response = requests.post(auth_url, json=body)
token = response.json()["token"]
auth_token = "Bearer " + token

In [10]:
protocolo_url = urljoin(api_url, "/protocolo?nested=true")
headers = {"Authorization": auth_token}

response = requests.get(protocolo_url, headers=headers)
protocolo = response.json()
protocolo = pd.DataFrame(protocolo)
protocolo.head(2)

id                                   titulo_protocolo cor_agenda  \
0   4  ENSAIO CLÍNICO FASE 2, RANDOMIZADO, DUPLO-CEGO...    #330eb9   
1   5  ENSAIO CLÍNICO FASE 2, RANDOMIZADO, DUPLO-CEGO...    #330eb9   

  numero_protocolo apelido_protocolo  coordenador     pi  pesquisador_backup  \
0    BTK-COV-202BR               BTK        261.0  306.0               210.0   
1    BTK-COV-202BR               BTK        310.0  199.0               227.0   

   co_pessoa_regulatorio             data_cadastro  ...  \
0                  214.0  2021-04-05T18:37:50.000Z  ...   
1                  214.0  2021-07-07T12:28:33.000Z  ...   

                 dados_aprovacao_cep  \
0  {'id': 60, 'ds_descricao': 'Sim'}   
1  {'id': 60, 'ds_descricao': 'Sim'}   

                               dados_co_centro  \
0       {'id': 3, 'descricao': 'Leforte HMCG'}   
1  {'id': 2, 'descricao': 'Leforte Liberdade'}   

                                            PessoaPI  \
0  {'id': 306, 'ds_nome': 'Dr. Carlos Augusto Qua...   
1  {'id': 199, 'ds_nome': 'Dra. Caroline Cândida ...   

                                   PessoaCoordenador  \
0            {'id': 261, 'ds_nome': 'Giovana Rosas'}   
1  {'id': 310, 'ds_nome': 'Isabel Cristina dos Sa...   

                                   PessoaPesquisador  \
0                                               None   
1  {'id': 227, 'ds_nome': 'Viviane Santana da Sil...   

                                   PessoaRegulatorio  \
0  {'id': 214, 'ds_nome': 'Priscila do Prado Costa'}   
1  {'id': 214, 'ds_nome': 'Priscila do Prado Costa'}   

                             fase_pesquisa  \
0  {'id': 123, 'ds_descricao': 'Fase III'}   
1   {'id': 122, 'ds_descricao': 'Fase II'}   

                                     status  \
0  {'id': 296, 'ds_descricao': 'Concluído'}   
1  {'id': 296, 'ds_descricao': 'Concluído'}   

                               tipo_iniciativa  \
0  {'id': 506, 'ds_descricao': 'Patrocinador'}   
1  {'id': 506, 'ds_descricao': 'Patrocinador'}   

                                   nome_patrocinador  
0  {'id': 723, 'ds_descricao': 'Sorrento Therapeu...  
1  {'id': 723, 'ds_descricao': 'Sorrento Therapeu...  

[2 rows x 124 columns]

In [12]:
protocolo_1 = protocolo[
    protocolo["status"].apply(lambda x: x['ds_descricao'] in ["Em Andamento", "Recrutamento Aberto"])
]

protocolo_1

id                                   titulo_protocolo cor_agenda  \
69     115  Um estudo multicêntrico, randomizado, duplo-ce...    #00d6d3   
89     141  Um Protocolo Multicêntrico de Fase 2/3, Random...    #000000   
212    300  Um estudo multicêntrico randomizado, duplo-ceg...    #e7bbf7   
227    322  Estudo Randomizado, Duplo-cego, Controlado por...    #000000   
326    447  Estudo clínico randomizado controlado por plac...    #d8eeea   
983   1182  Estudo clínico randomizado controlado por plac...    #6e00f5   
1015  1214                          Sinopse – Heparina Bovina    #000000   
1106  1311                                          Seminário    #000000   
1312  1519                                        OLMESARTANA    #000000   
1334  1542                       Advisory Board - Semaglutida    #000000   
1434  1643                                                       #000000   

                  numero_protocolo              apelido_protocolo  \
69                   CKJX839B12302          VICTORION 2 - PREVENT   
89           Parexel Number 232298                         GALAXI   
212                  CKJX839D12302            VICTORION-1-PREVENT   
227                CNTO1959CRD3004                        Graviti   
326                    MK0616-015                     MK0616-015    
983                    MK0616-015                     MK0616-015    
1015     Sinopse – Heparina Bovina      Sinopse – Heparina Bovina   
1106                     Seminário             Seminário Cardinal   
1312                   OLMESARTANA  Projeto Eurofarma Olmesartana   
1334  Advisory Board - Semaglutida   Advisory Board - Semaglutida   
1434                                Projeto DDCM / DPOC – Academy   

      coordenador      pi  pesquisador_backup  co_pessoa_regulatorio  \
69         2829.0   242.0                 NaN                  214.0   
89         2622.0  2384.0              2622.0                  214.0   
212        2829.0   242.0                 NaN                 2631.0   
227        2622.0  2384.0                 NaN                  214.0   
326        2829.0   266.0                 NaN                 2631.0   
983         239.0   256.0                 NaN                 2631.0   
1015          NaN     NaN                 NaN                    NaN   
1106          NaN     NaN                 NaN                    NaN   
1312          NaN     NaN                 NaN                    NaN   
1334          NaN     NaN                 NaN                    NaN   
1434          NaN     NaN                 NaN                    NaN   

                 data_cadastro  ...                dados_aprovacao_cep  \
69    2021-11-30T13:52:55.000Z  ...  {'id': 60, 'ds_descricao': 'Sim'}   
89    2022-02-15T17:41:41.000Z  ...  {'id': 60, 'ds_descricao': 'Sim'}   
212   2022-07-12T11:44:58.000Z  ...  {'id': 60, 'ds_descricao': 'Sim'}   
227   2022-08-05T12:20:38.000Z  ...  {'id': 60, 'ds_descricao': 'Sim'}   
326   2023-03-13T08:57:22.000Z  ...  {'id': 60, 'ds_descricao': 'Sim'}   
983   2023-10-17T16:04:51.000Z  ...  {'id': 60, 'ds_descricao': 'Sim'}   
1015  2023-10-23T14:31:04.000Z  ...                               None   
1106  2024-01-22T11:08:28.000Z  ...                               None   
1312  2024-03-25T15:59:26.000Z  ...                               None   
1334  2024-04-22T20:04:14.000Z  ...                               None   
1434  2024-06-06T17:20:58.000Z  ...                               None   

                                        dados_co_centro  \
69               {'id': 3, 'descricao': 'Leforte HMCG'}   
89       {'id': 4, 'descricao': 'Santa Casa de Santos'}   
212              {'id': 3, 'descricao': 'Leforte HMCG'}   
227      {'id': 4, 'descricao': 'Santa Casa de Santos'}   
326              {'id': 3, 'descricao': 'Leforte HMCG'}   
983   {'id': 5, 'descricao': 'Maternidade e Cirurgia...   
1015                 {'id': 54, 'descricao': 'Academy'}   
1106                 {'id': 54, 'descric

In [13]:
centros = protocolo_1[['id','apelido_protocolo', 'numero_protocolo','co_externo','apelido_centro']].copy()
centros

id              apelido_protocolo              numero_protocolo  \
69     115          VICTORION 2 - PREVENT                 CKJX839B12302   
89     141                         GALAXI         Parexel Number 232298   
212    300            VICTORION-1-PREVENT                 CKJX839D12302   
227    322                        Graviti               CNTO1959CRD3004   
326    447                    MK0616-015                    MK0616-015    
983   1182                    MK0616-015                    MK0616-015    
1015  1214      Sinopse – Heparina Bovina     Sinopse – Heparina Bovina   
1106  1311             Seminário Cardinal                     Seminário   
1312  1519  Projeto Eurofarma Olmesartana                   OLMESARTANA   
1334  1542   Advisory Board - Semaglutida  Advisory Board - Semaglutida   
1434  1643  Projeto DDCM / DPOC – Academy                                 

         co_externo                                     apelido_centro  
69    CKJX839B12302               VICTORION 2 - PREVENT - Leforte HMCG  
89            PT141                      GALAXI - Santa Casa de Santos  
212            None                 VICTORION-1-PREVENT - Leforte HMCG  
227            None                     Graviti - Santa Casa de Santos  
326            None                         MK0616-015  - Leforte HMCG  
983            None  MK0616-015  - Maternidade e Cirurgia Nossa Sen...  
1015           None                Sinopse – Heparina Bovina - Academy  
1106           None                       Seminário Cardinal - Academy  
1312           None            Projeto Eurofarma Olmesartana - Academy  
1334           None             Advisory Board - Semaglutida - Academy  
1434           None            Projeto DDCM / DPOC – Academy - Academy

In [14]:
participantes_url = urljoin(api_url, "/participantes?nested=true")
headers = {"Authorization": auth_token}

response = requests.get(participantes_url, headers=headers)
participantes = response.json()
participantes = pd.DataFrame(participantes)

participantes.head(2)

id  co_protocolo id_participante id_instituicao  nome  status_participante  \
0  54             4            1001        5684766  None                  308   
1  56             4            1002        5689356  None                  308   

  iniciais contatos data_nascimento observacoes  ... co_participante  \
0     None     None            None        None  ...              54   
1     None     None            None        None  ...              56   

                         dados_protocolo  \
0  {'id': 4, 'apelido_protocolo': 'BTK'}   
1  {'id': 4, 'apelido_protocolo': 'BTK'}   

                               dados_status                 dados_braco  \
0  {'id': 308, 'ds_descricao': 'Concluído'}  {'id': 1, 'nome': 'Único'}   
1  {'id': 308, 'ds_descricao': 'Concluído'}  {'id': 1, 'nome': 'Único'}   

                                    dados_voluntario  \
0  {'id': 54, 'iniciais': 'A.M.G', 'ds_nome': 'An...   
1  {'id': 55, 'iniciais': 'M.P.S', 'ds_nome': 'Ma...   

                                     Status                       Braco  \
0  {'id': 308, 'ds_descricao': 'Concluído'}  {'id': 1, 'nome': 'Único'}   
1  {'id': 308, 'ds_descricao': 'Concluído'}  {'id': 1, 'nome': 'Único'}   

                                          Voluntario  \
0  {'id': 54, 'iniciais': 'A.M.G', 'ds_nome': 'An...   
1  {'id': 55, 'iniciais': 'M.P.S', 'ds_nome': 'Ma...   

                                 ParticipantesVisita  \
0  [{'id': 639, 'co_participante': 54, 'co_visita...   
1  [{'id': 640, 'co_participante': 56, 'co_visita...   

                            ParticipanteUltimaVisita  
0  {'id': 54, 'co_voluntario': 54, 'ultima_visita...  
1  {'id': 56, 'co_voluntario': 55, 'ultima_visita...  

[2 rows x 35 columns]

In [15]:
partic = participantes[["id_participante", "dados_protocolo", "dados_status", "Status"]]
partic

id_participante                                    dados_protocolo  \
0                   1001              {'id': 4, 'apelido_protocolo': 'BTK'}   
1                   1002              {'id': 4, 'apelido_protocolo': 'BTK'}   
2                   1003              {'id': 4, 'apelido_protocolo': 'BTK'}   
3                   1004              {'id': 4, 'apelido_protocolo': 'BTK'}   
4                   1005              {'id': 4, 'apelido_protocolo': 'BTK'}   
...                  ...                                                ...   
2399  MEDIDOR 149 - Q178  {'id': 2057, 'apelido_protocolo': 'Post Market...   
2400  MEDIDOR 150 - Q179  {'id': 2057, 'apelido_protocolo': 'Post Market...   
2401  MEDIDOR 151 - Q180  {'id': 2057, 'apelido_protocolo': 'Post Market...   
2402  MEDIDOR 152 - Q181  {'id': 2057, 'apelido_protocolo': 'Post Market...   
2403          1076012018  {'id': 655, 'apelido_protocolo': 'EASi-HF - 13...   

                                         dados_status  \
0            {'id': 308, 'ds_descricao': 'Concluído'}   
1            {'id': 308, 'ds_descricao': 'Concluído'}   
2     {'id': 310, 'ds_descricao': 'Falha de seleção'}   
3            {'id': 308, 'ds_descricao': 'Concluído'}   
4            {'id': 308, 'ds_descricao': 'Concluído'}   
...                                               ...   
2399         {'id': 308, 'ds_descricao': 'Concluído'}   
2400         {'id': 308, 'ds_descricao': 'Concluído'}   
2401         {'id': 308, 'ds_descricao': 'Concluído'}   
2402         {'id': 308, 'ds_descricao': 'Concluído'}   
2403              {'id': 11, 'ds_descricao': 'Ativo'}   

                                               Status  
0            {'id': 308, 'ds_descricao': 'Concluído'}  
1            {'id': 308, 'ds_descricao': 'Concluído'}  
2     {'id': 310, 'ds_descricao': 'Falha de seleção'}  
3            {'id': 308, 'ds_descricao': 'Concluído'}  
4            {'id': 308, 'ds_descricao': 'Concluído'}  
...                                               ...  
2399         {'id': 308, 'ds_descricao': 'Concluído'}  
2400         {'id': 308, 'ds_descricao': 'Concluído'}  
2401         {'id': 308, 'ds_descricao': 'Concluído'}  
2402         {'id': 308, 'ds_descricao': 'Concluído'}  
2403              {'id': 11, 'ds_descricao': 'Ativo'}  

[2404 rows x 4 columns]

In [16]:
partic.loc[:, 'dados_protocolo_id'] = partic['dados_protocolo'].apply(lambda x: x['id'] if x is not None else None)
partic.loc[:, 'dados_status_info'] = partic['dados_status'].apply(lambda x: x['ds_descricao'] if x is not None else None)
partic.loc[:, 'Status_id'] = partic['Status'].apply(lambda x: x['id'] if x is not None else None)
partic.rename(columns={'dados_protocolo_id': 'id'}, inplace=True)
partic

C:\Users\Gleice Barros\AppData\Local\Temp\ipykernel_20008\4214383424.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partic.loc[:, 'dados_protocolo_id'] = partic['dados_protocolo'].apply(lambda x: x['id'] if x is not None else None)
C:\Users\Gleice Barros\AppData\Local\Temp\ipykernel_20008\4214383424.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partic.loc[:, 'dados_status_info'] = partic['dados_status'].apply(lambda x: x['ds_descricao'] if x is not None else None)
C:\Users\Gleice Barros\AppData\L

id_participante                                    dados_protocolo  \
0                   1001              {'id': 4, 'apelido_protocolo': 'BTK'}   
1                   1002              {'id': 4, 'apelido_protocolo': 'BTK'}   
2                   1003              {'id': 4, 'apelido_protocolo': 'BTK'}   
3                   1004              {'id': 4, 'apelido_protocolo': 'BTK'}   
4                   1005              {'id': 4, 'apelido_protocolo': 'BTK'}   
...                  ...                                                ...   
2399  MEDIDOR 149 - Q178  {'id': 2057, 'apelido_protocolo': 'Post Market...   
2400  MEDIDOR 150 - Q179  {'id': 2057, 'apelido_protocolo': 'Post Market...   
2401  MEDIDOR 151 - Q180  {'id': 2057, 'apelido_protocolo': 'Post Market...   
2402  MEDIDOR 152 - Q181  {'id': 2057, 'apelido_protocolo': 'Post Market...   
2403          1076012018  {'id': 655, 'apelido_protocolo': 'EASi-HF - 13...   

                                         dados_status  \
0            {'id': 308, 'ds_descricao': 'Concluído'}   
1            {'id': 308, 'ds_descricao': 'Concluído'}   
2     {'id': 310, 'ds_descricao': 'Falha de seleção'}   
3            {'id': 308, 'ds_descricao': 'Concluído'}   
4            {'id': 308, 'ds_descricao': 'Concluído'}   
...                                               ...   
2399         {'id': 308, 'ds_descricao': 'Concluído'}   
2400         {'id': 308, 'ds_descricao': 'Concluído'}   
2401         {'id': 308, 'ds_descricao': 'Concluído'}   
2402         {'id': 308, 'ds_descricao': 'Concluído'}   
2403              {'id': 11, 'ds_descricao': 'Ativo'}   

                                               Status    id dados_status_info  \
0            {'id': 308, 'ds_descricao': 'Concluído'}     4         Concluído   
1            {'id': 308, 'ds_descricao': 'Concluído'}     4         Concluído   
2     {'id': 310, 'ds_descricao': 'Falha de seleção'}     4  Falha de seleção   
3            {'id': 308, 'ds_descricao': 'Concluído'}     4         Concluído   
4            {'id': 308, 'ds_descricao': 'Concluído'}     4         Concluído   
...                                               ...   ...               ...   
2399         {'id': 308, 'ds_descricao': 'Concluído'}  2057         Concluído   
2400         {'id': 308, 'ds_descricao': 'Concluído'}  2057         Concluído   
2401         {'id': 308, 'ds_descricao': 'Concluído'}  2057         Concluído   
2402         {'id': 308, 'ds_descricao': 'Concluído'}  2057         Concluído   
2403              {'id': 11, 'ds_descricao': 'Ativo'}   655             Ativo   

      Status_id  
0           308  
1           308  
2           310  
3           308  
4           308  
...         ...  
2399        308  
2400        308  
2401        308  
2402        308  
2403         11  

[2404 rows x 7 columns]

In [17]:
partic_centro = pd.merge(partic, centros, on="id", how="inner")
partic_centro[['Estudo', "Centro de Pesquisa"]] = partic_centro["apelido_centro"].str.rsplit("-", n=1, expand=True)
partic_centro

id_participante                                    dados_protocolo  \
0          4132-001  {'id': 115, 'apelido_protocolo': 'VICTORION 2 ...   
1          4132-002  {'id': 115, 'apelido_protocolo': 'VICTORION 2 ...   
2          4132-004  {'id': 115, 'apelido_protocolo': 'VICTORION 2 ...   
3          4132-005  {'id': 115, 'apelido_protocolo': 'VICTORION 2 ...   
4          4132-010  {'id': 115, 'apelido_protocolo': 'VICTORION 2 ...   
..              ...                                                ...   
404      0724-00022    {'id': 447, 'apelido_protocolo': 'MK0616-015 '}   
405      0724-00023    {'id': 447, 'apelido_protocolo': 'MK0616-015 '}   
406      0724-00024    {'id': 447, 'apelido_protocolo': 'MK0616-015 '}   
407      0724-00025    {'id': 447, 'apelido_protocolo': 'MK0616-015 '}   
408      0721-00169   {'id': 1182, 'apelido_protocolo': 'MK0616-015 '}   

                                        dados_status  \
0    {'id': 310, 'ds_descricao': 'Falha de seleção'}   
1                {'id': 11, 'ds_descricao': 'Ativo'}   
2                {'id': 11, 'ds_descricao': 'Ativo'}   
3                {'id': 11, 'ds_descricao': 'Ativo'}   
4    {'id': 310, 'ds_descricao': 'Falha de seleção'}   
..                                               ...   
404  {'id': 310, 'ds_descricao': 'Falha de seleção'}   
405  {'id': 710, 'ds_descricao': 'Retirada de TCLE'}   
406              {'id': 11, 'ds_descricao': 'Ativo'}   
407              {'id': 11, 'ds_descricao': 'Ativo'}   
408  {'id': 310, 'ds_descricao': 'Falha de seleção'}   

                                              Status    id dados_status_info  \
0    {'id': 310, 'ds_descricao': 'Falha de seleção'}   115  Falha de seleção   
1                {'id': 11, 'ds_descricao': 'Ativo'}   115             Ativo   
2                {'id': 11, 'ds_descricao': 'Ativo'}   115             Ativo   
3                {'id': 11, 'ds_descricao': 'Ativo'}   115             Ativo   
4    {'id': 310, 'ds_descricao': 'Falha de seleção'}   115  Falha de seleção   
..                                               ...   ...               ...   
404  {'id': 310, 'ds_descricao': 'Falha de seleção'}   447  Falha de seleção   
405  {'id': 710, 'ds_descricao': 'Retirada de TCLE'}   447  Retirada de TCLE   
406              {'id': 11, 'ds_descricao': 'Ativo'}   447             Ativo   
407              {'id': 11, 'ds_descricao': 'Ativo'}   447             Ativo   
408  {'id': 310, 'ds_descricao': 'Falha de seleção'}  1182  Falha de seleção   

     Status_id      apelido_protocolo numero_protocolo     co_externo  \
0          310  VICTORION 2 - PREVENT    CKJX839B12302  CKJX839B12302   
1           11  VICTORION 2 - PREVENT    CKJX839B12302  CKJX839B12302   
2           11  VICTORION 2 - PREVENT    CKJX839B12302  CKJX839B12302   
3           11  VICTORION 2 - PREVENT    CKJX839B12302  CKJX839B12302   
4          310  VICTORION 2 - PREVENT    CKJX839B12302  CKJX839B12302   
..         ...                    ...              ...            ...   
404        310            MK0616-015       MK0616-015            None   
405        710            MK0616-015       MK0616-015            None   
406         11            MK0616-015       MK0616-015            None   
407         11            MK0616-015       MK0616-015            None   
408        310            MK0616-015       MK0616-015            None   

                                        apelido_centro  \
0                 VICTORION 2 - PREVENT - Leforte HMCG   
1                 VICTORION 2 - PREVENT - Leforte HMCG   
2                 VICTORION 2 - PREVENT - Leforte HMCG   
3                 VICTORION 2 - PREVENT - Leforte HMCG   
4                 VICTORION 2 - PREVENT - Leforte HMCG   
..                                                 ...   
404                         MK0616-015  - Leforte HMCG   
405                         MK0616-015  - Leforte HMCG   
406                         MK0616-015  - Leforte HMCG   
407                     

In [18]:
dados_modelados = partic_centro.drop(columns = ['numero_protocolo', 'co_externo', 'dados_protocolo', 'Status', 'apelido_centro', 'Estudo'], axis=1)

dados_modelados

id_participante                                     dados_status    id  \
0          4132-001  {'id': 310, 'ds_descricao': 'Falha de seleção'}   115   
1          4132-002              {'id': 11, 'ds_descricao': 'Ativo'}   115   
2          4132-004              {'id': 11, 'ds_descricao': 'Ativo'}   115   
3          4132-005              {'id': 11, 'ds_descricao': 'Ativo'}   115   
4          4132-010  {'id': 310, 'ds_descricao': 'Falha de seleção'}   115   
..              ...                                              ...   ...   
404      0724-00022  {'id': 310, 'ds_descricao': 'Falha de seleção'}   447   
405      0724-00023  {'id': 710, 'ds_descricao': 'Retirada de TCLE'}   447   
406      0724-00024              {'id': 11, 'ds_descricao': 'Ativo'}   447   
407      0724-00025              {'id': 11, 'ds_descricao': 'Ativo'}   447   
408      0721-00169  {'id': 310, 'ds_descricao': 'Falha de seleção'}  1182   

    dados_status_info  Status_id      apelido_protocolo  \
0    Falha de seleção        310  VICTORION 2 - PREVENT   
1               Ativo         11  VICTORION 2 - PREVENT   
2               Ativo         11  VICTORION 2 - PREVENT   
3               Ativo         11  VICTORION 2 - PREVENT   
4    Falha de seleção        310  VICTORION 2 - PREVENT   
..                ...        ...                    ...   
404  Falha de seleção        310            MK0616-015    
405  Retirada de TCLE        710            MK0616-015    
406             Ativo         11            MK0616-015    
407             Ativo         11            MK0616-015    
408  Falha de seleção        310            MK0616-015    

                                    Centro de Pesquisa  
0                                         Leforte HMCG  
1                                         Leforte HMCG  
2                                         Leforte HMCG  
3                                         Leforte HMCG  
4                                         Leforte HMCG  
..                                                 ...  
404                                       Leforte HMCG  
405                                       Leforte HMCG  
406                                       Leforte HMCG  
407                                       Leforte HMCG  
408   Maternidade e Cirurgia Nossa Senhora do Rocio SA  

[409 rows x 7 columns]

In [19]:
def contar_pacientes(dados_modelados):
    resultado = (
        dados_modelados.groupby(["Centro de Pesquisa", "apelido_protocolo", "dados_status_info"])["id_participante"]
          .nunique() 
          .reset_index(name="qtd_pacientes")
    )
    return resultado

# def contar_pacientes(dados_modelados):
#     # Define os níveis do índice
#     idx = pd.MultiIndex.from_product(
#         [
#             dados_modelados["Centro de Pesquisa"].unique(),
#             dados_modelados["apelido_protocolo"].unique(),
#             dados_modelados["dados_status_info"].unique()
#         ],
#         names=["Centro de Pesquisa", "apelido_protocolo", "dados_status_info"]
#     )

#     # Agrupa, conta e reindexa para incluir zeros
#     resultado = (
#         dados_modelados
#         .groupby(["Centro de Pesquisa", "apelido_protocolo", "dados_status_info"])["id_participante"]
#         .nunique()
#         .reindex(idx, fill_value=0)
#         .reset_index(name="qtd_pacientes")
#     )

#     return resultado

In [20]:
resultado_completo = contar_pacientes(dados_modelados)
resultado_completo

Centro de Pesquisa      apelido_protocolo  \
0                                        Leforte HMCG            MK0616-015    
1                                        Leforte HMCG            MK0616-015    
2                                        Leforte HMCG            MK0616-015    
3                                        Leforte HMCG  VICTORION 2 - PREVENT   
4                                        Leforte HMCG  VICTORION 2 - PREVENT   
5                                        Leforte HMCG  VICTORION 2 - PREVENT   
6                                        Leforte HMCG  VICTORION 2 - PREVENT   
7                                        Leforte HMCG  VICTORION 2 - PREVENT   
8                                        Leforte HMCG  VICTORION 2 - PREVENT   
9                                        Leforte HMCG    VICTORION-1-PREVENT   
10                                       Leforte HMCG    VICTORION-1-PREVENT   
11                                       Leforte HMCG    VICTORION-1-PREVENT   
12   Maternidade e Cirurgia Nossa Senhora do Rocio SA            MK0616-015    
13   Maternidade e Cirurgia Nossa Senhora do Rocio SA            MK0616-015    
14   Maternidade e Cirurgia Nossa Senhora do Rocio SA            MK0616-015    
15   Maternidade e Cirurgia Nossa Senhora do Rocio SA            MK0616-015    
16                               Santa Casa de Santos                 GALAXI   
17                               Santa Casa de Santos                Graviti   
18                               Santa Casa de Santos                Graviti   

                dados_status_info  qtd_pacientes  
0                           Ativo              9  
1                Falha de seleção             15  
2                Retirada de TCLE              1  
3                           Ativo             68  
4                   Descontinuado              1  
5   Descontinuou (outros motivos)              1  
6                Falha de seleção             43  
7                           Morte              6  
8                Retirada de TCLE              6  
9                           Ativo             58  
10               Falha de seleção             28  
11               Retirada de TCLE              1  
12                          Ativo             73  
13               Falha de seleção             90  
14                          Morte              2  
15               Retirada de TCLE              4  
16                          Ativo              1  
17                          Ativo              1  
18               Falha de seleção              1

In [21]:
resultado_completo.rename(columns={'apelido_protocolo': 'Estudo', 'dados_status_info': 'Status do Participante', 'qtd_pacientes': 'Nº de Pacientes'}, inplace=True)
resultado_completo

Centro de Pesquisa                 Estudo  \
0                                        Leforte HMCG            MK0616-015    
1                                        Leforte HMCG            MK0616-015    
2                                        Leforte HMCG            MK0616-015    
3                                        Leforte HMCG  VICTORION 2 - PREVENT   
4                                        Leforte HMCG  VICTORION 2 - PREVENT   
5                                        Leforte HMCG  VICTORION 2 - PREVENT   
6                                        Leforte HMCG  VICTORION 2 - PREVENT   
7                                        Leforte HMCG  VICTORION 2 - PREVENT   
8                                        Leforte HMCG  VICTORION 2 - PREVENT   
9                                        Leforte HMCG    VICTORION-1-PREVENT   
10                                       Leforte HMCG    VICTORION-1-PREVENT   
11                                       Leforte HMCG    VICTORION-1-PREVENT   
12   Maternidade e Cirurgia Nossa Senhora do Rocio SA            MK0616-015    
13   Maternidade e Cirurgia Nossa Senhora do Rocio SA            MK0616-015    
14   Maternidade e Cirurgia Nossa Senhora do Rocio SA            MK0616-015    
15   Maternidade e Cirurgia Nossa Senhora do Rocio SA            MK0616-015    
16                               Santa Casa de Santos                 GALAXI   
17                               Santa Casa de Santos                Graviti   
18                               Santa Casa de Santos                Graviti   

           Status do Participante  Nº de Pacientes  
0                           Ativo                9  
1                Falha de seleção               15  
2                Retirada de TCLE                1  
3                           Ativo               68  
4                   Descontinuado                1  
5   Descontinuou (outros motivos)                1  
6                Falha de seleção               43  
7                           Morte                6  
8                Retirada de TCLE                6  
9                           Ativo               58  
10               Falha de seleção               28  
11               Retirada de TCLE                1  
12                          Ativo               73  
13               Falha de seleção               90  
14                          Morte                2  
15               Retirada de TCLE                4  
16                          Ativo                1  
17                          Ativo                1  
18               Falha de seleção                1

In [22]:
resultado_completo["Centro de Pesquisa"] = resultado_completo["Centro de Pesquisa"].str.strip().str.lower()

In [23]:
grupos_centros = {
    "resultado_hmcg": ['Leforte HMCG','Leforte Morumbi','Hospital Municipal de Barueri', 
                        'Leforte Liberdade', 'Clinica CardialMed','Hospital Pérola Byington'],
    "resultado_capibaribe": ["Hospital Capibaribe"],
    "resultado_iir": ['Hospital das Clínicas de Itajubá', 'Beaute Clinique', 'Hospital Salvalus',
                       'Consultório Lopes e Sartorelli', 'Clínica Berger','QualiVida Higienópolis',
                       'Endolap Saúde','Associação Brasileira de Esclerose Múltipla','Centro Vila Olimpia' ],
    "resultado_vivi": ['Hospital Pilar','Santa casa de São Paulo','Hospital São Francisco de Ribeirão Preto',
                       'Hospital Antônio Prudente','Clínica CardialMed','CLINAR - Clínica de Aparelhos Respiratórios',
                       'Santa Casa de Fortaleza','Hospital Vera Cruz','Hapvida','Unimed Brusque',
                       'Hospital São José das Doenças Infecciosas','Otorhinus Clínica Médica','Hospital São Francisco de Araraquara', 
                       'Maternidade Octaviano Neves','Clínica Infectologie','Hospital Teresa de Lisieux',
                       'Hospital RioMar de Belém','Hospital e Maternidade Eugênia Pinheiro',
                       'Hospital São Francisco Saúde','Hospital do Coração de Campinas','Aliança Cavernoma Brasil',
                       'Faculdade de Medicina de Ribeirão Preto - USP ','Centro Clínico Zona Sul '],
    "resultado_rocio": ['Maternidade e Cirurgia Nossa Senhora do Rocio SA'],
    "resultado_scs" : ['Santa Casa de Santos'],
}

In [24]:
resultado_completo["Centro de Pesquisa"] = (
    resultado_completo["Centro de Pesquisa"].str.strip().str.upper()
)

grupos_centros = {
    k: [c.strip().upper() for c in v]
    for k, v in grupos_centros.items()
}

In [26]:
emails_por_grupo = {
    "resultado_hmcg": [destinatario_hmcg],
    "resultado_capibaribe": [destinatario_capibaribe],
    "resultado_iir" : [destinatario_iir],
    "resultado_vivi": [destinatario_vivi],
    "resultado_rocio": [destinatario_rocio],
    "resultado_scs": [destinatario_scs]
}

def enviar_relatorio_html(df_grupo, lista_emails, assunto, css_estilo):
    if not lista_emails:
        print("Nenhum destinatário válido. Email não será enviado.")
        return

    tabela_html = df_grupo.to_html(index=False, escape=False)

    html = f"""
    <html>
    <head>
        <style>
        {css_estilo}
        </style>
    </head>
    <body>
        <p>Olá,</p>
        <p>Segue o relatório de participantes por Centro de Pesquisa e por Estudo.:</p>
        {tabela_html}
        <p>Os dados aqui compilados foram elaborados com informações disposiveis na Polo Trial.</p> 
        <p>Qualquer duvida, por favor, procurar o time BI-SVRI.</p>
    </body>
    </html>
    """

    msg = MIMEMultipart()
    msg['From'] = email_usuario
    msg['To'] = ", ".join(lista_emails)
    msg['Subject'] = assunto
    msg.attach(MIMEText(html, 'html'))

    servidor = smtplib.SMTP(smtp_server, email_port)
    servidor.starttls()
    servidor.login(email_usuario, email_senha)
    servidor.sendmail(email_usuario, lista_emails, msg.as_string())
    servidor.quit()



for nome_grupo, centros in grupos_centros.items():

    df_grupo = resultado_completo[resultado_completo["Centro de Pesquisa"].isin(centros)]

    globals()[nome_grupo] = df_grupo

    lista_emails = emails_por_grupo.get(nome_grupo, [])

    if df_grupo.empty:
        print(f"Atenção: grupo {nome_grupo} não possui dados para envio.")
    else:
        enviar_relatorio_html(df_grupo, lista_emails,
                              assunto=f"Relatório {nome_grupo}",
                              css_estilo=css_hover)
        print(f"Email enviado para o grupo {nome_grupo}: {lista_emails}")

Email enviado para o grupo resultado_hmcg: ['sitehmcg@svriglobal.com']
Atenção: grupo resultado_capibaribe não possui dados para envio.
Atenção: grupo resultado_iir não possui dados para envio.
Atenção: grupo resultado_vivi não possui dados para envio.
Email enviado para o grupo resultado_rocio: ['siterocio@svriglobal.com']
Email enviado para o grupo resultado_scs: ['sitescsantos@svriglobal.com']
